#### TOP 100

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

def crawl_oliveyoung_best(scroll_pause=1.5, scroll_count=4):
    # Chrome auto setup
    options = Options()
    # options.add_argument("--headless=new")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 1) 홈 화면 열기
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    time.sleep(2)

    # 2) BEST 탭 클릭 (홈 화면 상단 메뉴)
    try:
        best_button = driver.find_element(By.CSS_SELECTOR, "a[href*='getBestList']")
        best_button.click()
        time.sleep(2)
    except:
        print("BEST 메뉴 클릭 실패")
        driver.quit()
        return pd.DataFrame()

    # 3) 스크롤 다운 (상품 리스트 추가 로딩)
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

    # 4) 상품 카드 추출
    items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
    print("상품 개수:", len(items))

    data = []
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "p.tx_name").text
        except:
            name = None

        try:
            price = item.find_element(By.CSS_SELECTOR, "span.tx_cur").text
        except:
            price = None

        try:
            review = item.find_element(By.CSS_SELECTOR, "span.review_num").text
        except:
            review = None

        try:
            rating = item.find_element(By.CSS_SELECTOR, "span.review_point").text
        except:
            rating = None

        data.append({
            "name": name,
            "price": price,
            "review": review,
            "rating": rating
        })

    driver.quit()
    return pd.DataFrame(data)


df = crawl_oliveyoung_best()

print(df.head())
print("총 개수:", len(df))

df.to_csv("oliveyoung_best.csv", index=False)

상품 개수: 100
                                                name       price review rating
0  [2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+2/10+...    9,900원 ~   None       
1         [어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획     24,500원   None       
2  [어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (+30ml+...     24,700원   None       
3  [어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디션 (올영한정)/미니...  115,900원 ~   None       
4  [어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml 어워즈 ...     33,250원   None       
총 개수: 100


In [11]:
def get_reviews(prod_id, max_page=20):
    reviews = []

    for page in range(1, max_page+1):
        url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsEvalList.do?goodsNo={prod_id}&pageIdx={page}"
        r = requests.get(url, headers=headers)

        try:
            data = r.json()
        except:
            break

        eval_list = data.get("list", [])
        if not eval_list:
            break

        for rv in eval_list:
            reviews.append({
                "user": rv.get("userId"),
                "score": rv.get("evalScore"),
                "content": rv.get("evalContent"),
                "date": rv.get("regDt")
            })

    return pd.DataFrame(reviews)

#### NEW 크롤링

In [16]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def crawl_oliveyoung_new(scroll_pause=1.5, scroll_count=4):
    options = Options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 홈 접속
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    time.sleep(2)

    # NEW 버튼 클릭
    try:
        new_btn = driver.find_element(By.CSS_SELECTOR, "a[href*='getNewList']")
        new_btn.click()
    except:
        print("NEW 메뉴 클릭 실패")
        driver.quit()
        return pd.DataFrame()

    # Ajax 로딩되는 NEW 상품 리스트 등장할 때까지 기다림
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
        )
    except:
        print("NEW 상품 로딩 실패")
        driver.quit()
        return pd.DataFrame()

    time.sleep(1)

    # 스크롤
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

    # 상품 요소 추출
    items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
    print("상품 개수:", len(items))

    data = []
    for item in items:
        try: name = item.find_element(By.CSS_SELECTOR, "p.tx_name").text
        except: name = None
        try: price = item.find_element(By.CSS_SELECTOR, "span.tx_cur").text
        except: price = None
        try: review = item.find_element(By.CSS_SELECTOR, "span.review_num").text
        except: review = None
        try: rating = item.find_element(By.CSS_SELECTOR, "span.review_point").text
        except: rating = None

        data.append({
            "name": name,
            "price": price,
            "review": review,
            "rating": rating
        })

    driver.quit()
    return pd.DataFrame(data)

In [27]:
df_new = crawl_oliveyoung("https://www.oliveyoung.co.kr/store/main/getNewList.do")

상품 개수: 0


In [1]:
def get_driver():
    options = Options()
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )
    options.add_argument("--window-size=1500,1000")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )
    return driver

In [2]:
def crawl_new():
    driver = get_driver()

    # oy=1 로 강제 PC 페이지 유지
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do?oy=1")
    time.sleep(3)

    # NEW 버튼 클릭 (href 기반)
    new_btn = driver.find_element(By.CSS_SELECTOR, "a[href*='getNewList']")
    new_btn.click()
    time.sleep(3)

    # 상품 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
    )

    items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
    print("상품 개수:", len(items))

    driver.quit()

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


def get_driver():
    options = Options()

    # PC 환경 강제
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )
    options.add_argument("--window-size=1500,1000")

    # WebDriver 감지 우회
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # navigator.webdriver 우회
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )

    return driver

In [6]:
def crawl_oliveyoung_new(scroll_pause=1.5, scroll_count=4):
    driver = get_driver()

    # ✔ PC 버전 강제 페이지
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do?oy=1")
    time.sleep(3)

    # ✔ NEW(신상) 메뉴: href 기반으로만 존재함
    try:
        new_btn = driver.find_element(By.CSS_SELECTOR, "a[href*='getNewList']")
        new_btn.click()
        print("NEW 메뉴 클릭 성공")
    except:
        print("NEW 메뉴 클릭 실패 (href 방식만 가능)")
        driver.quit()
        return pd.DataFrame()

    # Ajax로 상품 리스트 로딩될 때까지 기다림
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
        )
    except:
        print("NEW 상품 로딩 실패")
        driver.quit()
        return pd.DataFrame()

    time.sleep(1)

    # 스크롤 다운
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

    # 상품 요소 추출
    items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
    print("상품 개수:", len(items))

    data = []
    for item in items:
        try: name = item.find_element(By.CSS_SELECTOR, "p.tx_name").text
        except: name = None
        try: price = item.find_element(By.CSS_SELECTOR, "span.tx_cur").text
        except: price = None
        try: review = item.find_element(By.CSS_SELECTOR, "span.review_num").text
        except: review = None
        try: rating = item.find_element(By.CSS_SELECTOR, "span.review_point").text
        except: rating = None

        data.append({
            "name": name,
            "price": price,
            "review": review,
            "rating": rating
        })

    driver.quit()
    return pd.DataFrame(data)

In [8]:
df_new = crawl_oliveyoung_new()
df_new.head()

NEW 메뉴 클릭 실패 (href 방식만 가능)


""


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd


def crawl_oliveyoung_tab(tab_text, scroll_pause=1.5, scroll_count=4):
    """
    tab_text : '오특', '랭킹', '세일', '기획전', '헬스+' 같은 상단 탭 이름
    """

    options = Options()
    # 필요하면 headless 켜기
    # options.add_argument("--headless=new")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 1) 메인 들어가기
    driver.get("https://www.oliveyoung.co.kr/store/main/main.do")
    time.sleep(2)

    # 2) 상단 탭 클릭 (텍스트로 찾기)
    try:
        tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.XPATH, f"//div[@id='gnb']//a[contains(., '{tab_text}')]")
            )
        )
        tab.click()
    except Exception as e:
        print(f"'{tab_text}' 탭 클릭 실패:", e)
        driver.quit()
        return pd.DataFrame()

    # 3) 상품 리스트 나타날 때까지 대기
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "ul.cate_prd_list > li")
            )
        )
    except:
        print("상품 리스트 로딩 실패")
        driver.quit()
        return pd.DataFrame()

    # 4) 스크롤 내려서 더 로딩
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

    # 5) 상품 카드 수집
    items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
    print(f"[{tab_text}] 상품 개수:", len(items))

    data = []
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "p.tx_name").text
        except:
            name = None

        try:
            price = item.find_element(By.CSS_SELECTOR, "span.tx_cur").text
        except:
            price = None

        try:
            review = item.find_element(By.CSS_SELECTOR, "span.review_num").text
        except:
            review = None

        try:
            rating = item.find_element(By.CSS_SELECTOR, "span.review_point").text
        except:
            rating = None

        data.append(
            {
                "name": name,
                "price": price,
                "review": review,
                "rating": rating,
                "tab": tab_text,
            }
        )

    driver.quit()
    return pd.DataFrame(data)

In [10]:
# 오특
df_ot = crawl_oliveyoung_tab("오특")

# 랭킹
df_rank = crawl_oliveyoung_tab("랭킹")

# 세일
df_sale = crawl_oliveyoung_tab("세일")

# 기획전
df_plan = crawl_oliveyoung_tab("기획전")

'오특' 탭 클릭 실패: Message: 
Stacktrace:
0   chromedriver                        0x000000010276aecc cxxbridge1$str$ptr + 2941512
1   chromedriver                        0x0000000102762b88 cxxbridge1$str$ptr + 2907908
2   chromedriver                        0x000000010227a2b0 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74020
3   chromedriver                        0x00000001022c188c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 366336
4   chromedriver                        0x0000000102302d54 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 633800
5   chromedriver                        0x00000001022b5ef0 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 318820
6   chromedriver                        0x000000010272e0c8 cxxbridge1$str$ptr + 2692164
7   chromedriver                        0x00000001027318dc cxxbridge1$str$ptr + 2706520
8   chromedriver                        0x000000010270e84c cxxbridge1$str$ptr

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1) 드라이버 생성
options = Options()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)
options.add_argument("--window-size=1500,1000")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# 2) 페이지 열기
driver.get("https://www.oliveyoung.co.kr/store/main/main.do?oy=1")
time.sleep(3)

# 3) DOM 앞 3000자 출력
html = driver.page_source
print(html[:3000])

<html lang="ko" class=" -webkit-"><head><script type="text/javascript" async="" src="https://analytics.tiktok.com/i18n/pixel/static/identify_999e0538.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-D4CXJXVDV8&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-PZZTG1SN65&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://analytics.tiktok.com/i18n/pixel/static/main.MTRhZTU1OWY5MQ.js" data-id="CMJ3CKJC77UBB48CH0CG"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-944484389&amp;cx=c&amp;gtm=4e5c21h1"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-672565820&amp;cx=c&amp;gtm=4e5c21h1"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-849939742&amp;cx=c&amp

In [14]:
print(driver.page_source[9000:18000])

10 19.0625ZM12.772 6.15396L13.8706 7.2525L11.1242 9.99885L13.8706 12.7452L12.772 13.8437L10.0257 11.0974L7.27933 13.8438L6.18079 12.7452L8.92715 9.99885L6.18079 7.2525L7.27933 6.15396L10.0257 8.90031L12.772 6.15396Z' fill='%23C9CDD2'/%3E%3C/g%3E%3Cdefs%3E%3CclipPath id='clip0_11079_39456'%3E%3Crect width='20' height='20' fill='white'/%3E%3C/clipPath%3E%3C/defs%3E%3C/svg%3E") center/20px 20px no-repeat;text-indent:-9999px}.header_inner .search_box #searchSubmit{flex-shrink:0;width:40px;height:38px;background:url("data:image/svg+xml,%3Csvg width='20' height='20' viewBox='0 0 20 20' fill='none' xmlns='http://www.w3.org/2000/svg'%3E%3Cpath d='M8.44339 14.378C11.7226 14.3764 14.3796 11.7168 14.3779 8.43756C14.3763 5.15837 11.7167 2.50138 8.4375 2.50301C5.15831 2.50463 2.50132 5.16426 2.50294 8.44345C2.50457 11.7226 5.1642 14.3796 8.44339 14.378Z' stroke='%23131518' stroke-width='1.5'/%3E%3Cpath d='M16.9623 18.0436C17.2552 18.3365 17.7301 18.3365 18.023 18.0436C18.3159 17.7507 18.3159 17.275

#### 오특 여는 페이지

In [20]:
print("STATUS:", r.status_code)
print("HEADERS:", r.headers)
print("TEXT:", r.text[:5000])

STATUS: 403
HEADERS: {'Date': 'Thu, 04 Dec 2025 16:41:02 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'accept-ch': 'Sec-CH-UA-Bitness, Sec-CH-UA-Arch, Sec-CH-UA-Full-Version, Sec-CH-UA-Mobile, Sec-CH-UA-Model, Sec-CH-UA-Platform-Version, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Platform, Sec-CH-UA, UA-Bitness, UA-Arch, UA-Full-Version, UA-Mobile, UA-Model, UA-Platform-Version, UA-Platform, UA', 'cf-mitigated': 'challenge', 'critical-ch': 'Sec-CH-UA-Bitness, Sec-CH-UA-Arch, Sec-CH-UA-Full-Version, Sec-CH-UA-Mobile, Sec-CH-UA-Model, Sec-CH-UA-Platform-Version, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Platform, Sec-CH-UA, UA-Bitness, UA-Arch, UA-Full-Version, UA-Mobile, UA-Model, UA-Platform-Version, UA-Platform, UA', 'cross-origin-embedder-policy': 'require-corp', 'cross-origin-opener-policy': 'same-origin', 'cross-origin-resource-policy': 'same-origin', 'origin-agent-cluster': '?1', 'permissions-policy': 'accelerometer=(),browsing-

In [17]:
def crawl_hotdeal(scroll_pause=1.5, scroll_count=4):
    options = Options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get("https://www.oliveyoung.co.kr/store/main/getHotdealList.do")
    time.sleep(2)

    # 스크롤
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

    # 상품 li
    items = driver.find_elements(By.CSS_SELECTOR, "ul.prod-list li")
    print("오특 상품 개수:", len(items))

    data = []
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "a.goodsList__info .prod-name").text
        except:
            name = None

        try:
            price = item.find_element(By.CSS_SELECTOR, "a.goodsList__info .price-amount").text
        except:
            price = None

        try:
            review = item.find_element(By.CSS_SELECTOR, "a.goodsList__info .review").text
        except:
            review = None

        try:
            rating = item.find_element(By.CSS_SELECTOR, "a.goodsList__info .rating").text
        except:
            rating = None

        data.append({
            "name": name,
            "price": price,
            "review": review,
            "rating": rating
        })

    driver.quit()
    return pd.DataFrame(data)

In [18]:
df_hot = crawl_hotdeal()
print(df_hot.head())
print("총 개수:", len(df_hot))
df_hot.to_csv("oliveyoung_hotdeal.csv", index=False)

오특 상품 개수: 35
   name price review rating
0  None  None   None   None
1  None  None   None   None
2  None  None   None   None
3  None  None   None   None
4  None  None   None   None
총 개수: 35


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
import pandas as pd


def crawl_hotdeal_oliveyoung(scroll_pause=1.5, scroll_count=4):
    # 1) 드라이버 세팅
    options = Options()
    # 필요하면 헤드리스 사용
    # options.add_argument("--headless=new")
    options.add_argument("--window-size=1400,900")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    try:
        # 2) 오특 페이지 진입
        url = "https://www.oliveyoung.co.kr/store/main/getHotdealList.do"
        driver.get(url)

        # 주요 텍스트가 뜰 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[contains(text(),'오늘의 특가')]")
            )
        )

        # 3) 아래로 여러 번 스크롤해서 전체 로딩
        for _ in range(scroll_count):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause)

        # 4) 상품 상세로 가는 링크(anchor)들만 수집
        #   - 특징: href 에 'goods/getGoodsDetail' 이 들어감
        prod_links = driver.find_elements(
            By.CSS_SELECTOR,
            "a[href*='goods/getGoodsDetail']"
        )

        print("오특 상품 anchor 개수:", len(prod_links))

        data = []
        for a in prod_links:
            raw = a.text.strip().replace("\n", " ")
            if not raw:
                continue  # 텍스트 없는 앵커는 스킵

            href = a.get_attribute("href")

            # 텍스트 안에서 가격 패턴(숫자+원) 전부 추출
            prices = re.findall(r"([\d,]+)\s*원", raw)

            # 첫 번째 가격이 나타나기 전까지를 '상품명'으로 취급
            name = raw
            m = re.search(r"([\d,]+)\s*원", raw)
            if m:
                name = raw[:m.start()].strip()

            # 간단히 할인가(첫 번째), 정가(마지막)로도 나눠 둠
            sale_price = prices[0] if prices else None
            orig_price = prices[-1] if len(prices) >= 2 else None

            data.append(
                {
                    "name": name,
                    "sale_price": sale_price,
                    "orig_price": orig_price,
                    "all_prices": " / ".join(prices),
                    "raw_text": raw,
                    "url": href,
                }
            )

        df = pd.DataFrame(data)

    finally:
        driver.quit()

    return df

In [22]:
df_hot = crawl_hotdeal_oliveyoung()
print(df_hot.head())
print("총 개수:", len(df_hot))

df_hot.to_csv("oliveyoung_hotdeal.csv", index=False)

오특 상품 anchor 개수: 70
                                                name sale_price orig_price  \
0  [12/5 하루특가] 아렌시아 그린티 LHA 딥 포어 떡솝 클렌저 150g 어워즈 ...     16,900     28,000   
1        [12/5 하루특가/8년연속1위] 케어플러스 상처 커버 스팟 패치 (102매)      5,200      6,300   
2  [12/5 하루특가] 미쟝센 컬링에센스 어워즈 한정기획 230ML*2 (볼륨/내추럴...     13,700     30,000   
3     [12/5 하루특가] 라운드랩 자작나무 수분 선크림 50ml 1+1 어워즈 한정기획     23,400     25,000   
4  [12/5 하루특가/어워즈특가] 콜레올로지 컷팅 젤리 10포 어워즈 한정기획 (+컷...     17,500     23,900   

        all_prices                                           raw_text  \
0  16,900 / 28,000  [12/5 하루특가] 아렌시아 그린티 LHA 딥 포어 떡솝 클렌저 150g 어워즈 ...   
1    5,200 / 6,300  [12/5 하루특가/8년연속1위] 케어플러스 상처 커버 스팟 패치 (102매) 5,...   
2  13,700 / 30,000  [12/5 하루특가] 미쟝센 컬링에센스 어워즈 한정기획 230ML*2 (볼륨/내추럴...   
3  23,400 / 25,000  [12/5 하루특가] 라운드랩 자작나무 수분 선크림 50ml 1+1 어워즈 한정기획...   
4  17,500 / 23,900  [12/5 하루특가/어워즈특가] 콜레올로지 컷팅 젤리 10포 어워즈 한정기획 (+컷...   

                                                 url  
0  https://www.ol

####  세일 

In [27]:
def crawl_sale_all(max_pages=50, scroll_pause=1.5, scroll_count=3):
    options = Options()
    options.add_argument("--window-size=1400,900")
    # options.add_argument("--headless=new")

    service = Service(ChromeDriverManager().install())

    all_data = []

    for page in range(1, max_pages + 1):

        driver = webdriver.Chrome(service=service, options=options)
        try:
            url = f"https://www.oliveyoung.co.kr/store/main/getSaleList.do?pageIdx={page}"
            driver.get(url)

            # 첫 상품이 뜰 때까지 대기
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list > li"))
                )
            except:
                break  # 더 이상 페이지 없음

            # 스크롤
            for _ in range(scroll_count):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(scroll_pause)

            items = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list > li")
            print(f"{page}페이지 상품 개수:", len(items))

            # 더 이상 상품 없으면 종료
            if len(items) == 0:
                break

            for li in items:
                try:
                    name = li.find_element(By.CSS_SELECTOR, ".prd_name").text.strip()
                except:
                    continue

                try:
                    price = li.find_element(By.CSS_SELECTOR, ".tx_cur").text.strip()
                except:
                    price = None

                try:
                    orig_price = li.find_element(By.CSS_SELECTOR, ".tx_org").text.strip()
                except:
                    orig_price = None

                try:
                    url = li.find_element(
                        By.CSS_SELECTOR, "a[href*='goods/getGoodsDetail']"
                    ).get_attribute("href")
                except:
                    url = None

                all_data.append({
                    "page": page,
                    "name": name,
                    "price": price,
                    "orig_price": orig_price,
                    "url": url
                })

        finally:
            driver.quit()

    return pd.DataFrame(all_data)

In [28]:
df_sale_all = crawl_sale_all()
print(df_sale_all.head())
print("총 개수:", len(df_sale_all))

df_sale_all.to_csv("oliveyoung_sale_all.csv", index=False)

1페이지 상품 개수: 24
2페이지 상품 개수: 24
3페이지 상품 개수: 24
4페이지 상품 개수: 24
5페이지 상품 개수: 24
6페이지 상품 개수: 24
7페이지 상품 개수: 24
8페이지 상품 개수: 24
9페이지 상품 개수: 24
10페이지 상품 개수: 24
11페이지 상품 개수: 24
12페이지 상품 개수: 24
13페이지 상품 개수: 24
14페이지 상품 개수: 24
15페이지 상품 개수: 24
16페이지 상품 개수: 24
17페이지 상품 개수: 24
18페이지 상품 개수: 24
19페이지 상품 개수: 24
20페이지 상품 개수: 24
21페이지 상품 개수: 24
22페이지 상품 개수: 24
23페이지 상품 개수: 24
24페이지 상품 개수: 24
25페이지 상품 개수: 24
26페이지 상품 개수: 24
27페이지 상품 개수: 24
28페이지 상품 개수: 24
29페이지 상품 개수: 24
30페이지 상품 개수: 24
31페이지 상품 개수: 24
32페이지 상품 개수: 24
33페이지 상품 개수: 24
34페이지 상품 개수: 24
35페이지 상품 개수: 24
36페이지 상품 개수: 24
37페이지 상품 개수: 24
38페이지 상품 개수: 24
39페이지 상품 개수: 24
40페이지 상품 개수: 24
41페이지 상품 개수: 24
42페이지 상품 개수: 24
43페이지 상품 개수: 24
44페이지 상품 개수: 24
45페이지 상품 개수: 24
46페이지 상품 개수: 24
47페이지 상품 개수: 24
48페이지 상품 개수: 24
49페이지 상품 개수: 24
50페이지 상품 개수: 24
   page                                               name       price  \
0     1  메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...    9,900원 ~   
1     1    일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획   

In [29]:
df_sale_all.to_csv("oliveyoung_sale_all.csv", index=False, encoding="utf-8-sig")